In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError, RootMeanSquaredError
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.config.experimental import list_physical_devices, set_virtual_device_configuration, VirtualDeviceConfiguration

gpus = list_physical_devices('GPU')
set_virtual_device_configuration(gpus[0], [VirtualDeviceConfiguration(memory_limit=1200)])
print(len(gpus), "Physical GPUs")   

2023-04-09 13:14:31.299774: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


1 Physical GPUs


2023-04-09 13:14:34.093920: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-09 13:14:34.105320: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-09 13:14:34.195955: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-09 13:14:34.196553: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.83GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 462.00GiB/s
2023-04-09 13:14:34.196587: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-09 13:14:34.287925: I tensorflow/stream_executor/platfo

In [2]:
df = pd.read_csv("Data/scaled_xyzlog_without_GRBs.csv", index_col=0) # data (hopefuly without grbs)
df

/home/mchobola/anaconda3/envs/envi/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,time,x,y,z,log_cps
0,481.489854,0.351577,-0.470408,-0.809389,1.981139
1,481.489901,0.402253,-0.651949,-0.642771,1.985651
2,481.489947,0.402425,-0.804490,-0.436865,1.984527
3,481.489993,0.359273,-0.910516,-0.204654,2.024280
4,481.490039,0.284601,-0.957816,0.039879,1.973128
...,...,...,...,...,...
2054802,719.491781,-0.080000,0.068985,0.994405,2.120574
2054803,719.491793,-0.125928,0.111252,0.985782,2.130334
2054804,719.491804,-0.170043,0.154518,0.973247,2.152288
2054805,719.491816,-0.212694,0.197985,0.956851,2.176091


In [3]:
def df_to_X_y(df, windowSize=128):
    dfNumpy = df.to_numpy()
    X,y = [], []
    for i in range(len(dfNumpy)-windowSize):
        if abs(dfNumpy[i+windowSize][0] - (windowSize*(dfNumpy[i-1][0]-dfNumpy[i][0])+dfNumpy[i][0])) < 1:
            X.append(dfNumpy[i:i+windowSize])
            y.append(dfNumpy[i+windowSize][-1])
    return np.array(X), np.array(y)

def lossPlot(history,name,figsize=(5,3)):
    t_loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.figure(figsize=figsize)
    plt.plot(t_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.yscale("log")
    plt.savefig('loss_{}.png'.format(name))
    
def isModelGood(model,name):
    grbs = glob.glob("Data/scaled_xyzlog/*.csv")
    n_grbs = len(grbs)
    fig, axs = plt.subplots(n_grbs,5,figsize=(30,3*n_grbs))
    for n , grb in enumerate(grbs):
        df = pd.read_csv(grb, index_col=0)
        num_df = df.to_numpy()
        df = df.drop(['0'],axis=1)
        y_pred0 = []
        X0 = df[:windowSize].copy().to_numpy()
        for i in range(len(df)-windowSize):
            pred = model.predict(X0.reshape(1,windowSize,4))[0]
            y_pred0.append(pred)
            X0 = df[i+1:i+windowSize+1].copy().to_numpy()
            X0[:,-1] = pred
            
        true_lc = 10**(np.array(num_df[windowSize:,-1]))
        pred_lc = 10**(np.array(y_pred0).T[0])
        
        axs[n,0].hist(true_lc-pred_lc)
        
        axs[n,1].plot(true_lc)
        axs[n,1].plot(pred_lc)
        
        axs[n,2].plot(true_lc[:10])
        axs[n,2].plot(pred_lc[:10]) 
        
        axs[n,3].plot(true_lc[-64:])
        axs[n,3].plot(pred_lc[-64:])
        
        axs[n,4].plot((true_lc-pred_lc)/true_lc)
    fig.savefig('LSTM_{}.png'.format(name))


def training_LSTM(df, windowSize,n_neuro_LSTM,n_neuro_dense,n_layers,n_output,loss,metrics):
    fileName = '{}-{}-{}-{}-{}-{}'.format(windowSize,n_neuro_LSTM,n_neuro_dense,n_layers,loss.__class__.__name__,metrics.__class__.__name__)
    print(fileName)
    X1, y1 = df_to_X_y(df, windowSize)
    splitIndex = int(len(df) * 0.8)
    X_train, y_train, X_test, y_test = X1[0:splitIndex,:,1:],y1[0:splitIndex],X1[splitIndex:,:,1:],y1[splitIndex:]
    
    model = Sequential()
    model.add(InputLayer((windowSize,4)))
    model.add(LSTM(n_neuro_LSTM))
    for i in range(n_layers):
        model.add(Dense(n_neuro_dense, 'relu'))
    model.add(Dense(n_output, 'linear'))
    
    model.compile(loss=loss,
                   optimizer=Adam(learning_rate=0.01),
                   metrics=metrics)

    #EarlyStopping(monitor='val_loss', patience=5)
    #ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=3,verbose=True,min_lr=0.00001)
    callback = [ModelCheckpoint("models/LSTM_{}.h5".format(fileName), 
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True, 
                                 mode='min'),
               TensorBoard(log_dir="Tensor_logs")]

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,callbacks=[callback])
    lossPlot(history,fileName)
    isModelGood(model,fileName)

In [ ]:
# def training_LSTM(df, windowSize,n_neuro_LSTM,n_neuro_dense,n_layers,n_output,loss,metrics):

win = [128]
n_neuro_LSTM = [64]
n_neuro_dense = [16,32]
n_layers = [7,5,3]
losses = [MeanSquaredError()]#,MeanAbsoluteError()]
metrics = [MeanSquaredError()] #MeanAbsoluteError()
for window in win:
    for n in n_neuro_LSTM:
        for nn in n_neuro_dense:
            for nnn in n_layers:
                for loss in losses:
                    for metric in metrics:
                        training_LSTM(df,window,n,nn,nnn,1,loss,metric)

128-64-16-7-MeanSquaredError-MeanSquaredError
